In [1]:
import tensorflow as tf
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
with open("war_text.pickle", "rb") as file:
    war_text = pickle.load(file)
    
len(war_text)

726

In [3]:
all_text = ''.join(war_text)

id2char = sorted(set(all_text))
char2id = {c:i for i, c in enumerate(id2char)}
vocab_size = len(id2char)

In [4]:
id2char = np.array(id2char)

In [5]:
# batch_size = 64
# seq_length = 200

# def text_to_int(txt):
#     return np.array([char2id[c] for c in txt])

# def article_to_example(art):
#     curr = art + '\n'*((-len(art))%(seq_length+1))
#     for i in range(len(curr)//(seq_length+1)):
#         yield text_to_int(curr[i*(seq_length+1):(i+1)*(seq_length+1)-1]), text_to_int(curr[i*(seq_length+1)+1:(i+1)*(seq_length+1)])

# def generator():
#     batch = []
#     for art in war_text:
#         for e in article_to_example(art):
#             batch.append(e)
#             if (len(batch)==batch_size):
#                 yield tuple(zip(*batch))
#                 batch = []
        
# check = list(generator())[:4]
# for a in check:
#     print(id2char[a[0][0]], id2char[a[1][0]])

In [6]:
# dataset = tf.data.Dataset.from_generator(
#     generator,
#     (tf.int16, tf.int16),
#     ((batch_size, seq_length), (batch_size, seq_length))
# )
# dataset

In [7]:
all_text_as_int = np.array([char2id[c] for c in all_text])

In [8]:
seq_length = 200
examples_per_epoch = len(all_text)

char_dataset = tf.data.Dataset.from_tensor_slices(all_text_as_int)

for i in char_dataset.take(10):
    print(id2char[i.numpy()])

К
р
и
м
с
к
а
т
а
 


In [9]:
id2char = np.array(id2char)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(id2char[item.numpy()])))

'Кримската война (октомври 1853\xa0– февруари 1856) е военен конфликт между Русия и съюз между Османската империя, Франция, Британската империя и Сардинското кралство.\n\nВойната е част от дълъг конфликт меж'
'ду Великите европейски сили за влияние върху териториите на западащата Османска империя. Основната част на конфликта се развива на Кримския полуостров, но военни действия се провеждат и на Балканите, в'
' Кавказкия регион, Балтийско море и Тихия океан.\n\nВ Русия е известна като Източната война (на руски: Восточная война), в Англия като Руската война (Russian war), в земите на Османската империя – като С'
'евастополското мурабе.\n\nКримската война се разглежда като първата „модерна война“ с използване на технологични новости като железници и телеграф. Тя е и първата война, широко документирана с фотография'
'.Причини за конфликта\nКъм средата на 19 век Англия и Франция изместват Русия от близкоизточните пазари и поставят Османската империя под свое влияние. Император Николай I не

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(id2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(id2char[target_example.numpy()])))

Input data:  'Кримската война (октомври 1853\xa0– февруари 1856) е военен конфликт между Русия и съюз между Османската империя, Франция, Британската империя и Сардинското кралство.\n\nВойната е част от дълъг конфликт ме'
Target data: 'римската война (октомври 1853\xa0– февруари 1856) е военен конфликт между Русия и съюз между Османската империя, Франция, Британската империя и Сардинското кралство.\n\nВойната е част от дълъг конфликт меж'


In [12]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 200), (64, 200)), types: (tf.int32, tf.int32)>

In [13]:
embedding_dim = 64
rnn_units = 1024

def build_model(batch_size=64):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, 
                                        embedding_dim, 
                                        batch_input_shape=[batch_size, None]))
    model.add(tf.keras.layers.LSTM(rnn_units, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 64)            21696     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4460544   
_________________________________________________________________
dense (Dense)                (64, None, 339)           347475    
Total params: 4,829,715
Trainable params: 4,829,715
Non-trainable params: 0
_________________________________________________________________


In [14]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [15]:
import os

checkpoint_dir = "./war_char"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = os.path.join(checkpoint_dir, "ckpt_{epoch}"),
    save_weights_only=True
)

In [16]:
def generate_text(model, start_string, num_generate = 200, temperature = 1.0):

    input_eval = [char2id[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(id2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [17]:
class PrintGenText(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if (epoch>0):
            curr_model = build_model(1)
            curr_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
            curr_model.build(tf.TensorShape([1, None]))
            print(generate_text(curr_model, "В началото на", 200))
        
gen_callback = PrintGenText()

In [18]:
history = model.fit(dataset, shuffle=True, epochs=50, callbacks = [checkpoint_callback, gen_callback])

Train for 181 steps
Epoch 1/50
181/181 [==============================] - 38s 209ms/step - loss: 3.0825
В началото нап райm вне ола онаа и ра-найто, а. иотиво ларитря ностот, ени ка Хамечвбадотери, и йс деле, о5 a95.
р#0 Треллятиян о ная не 1о Плеатоде
се 13зос., о даподет ет дене хувика на, даларамомисте ослацадока
Epoch 2/50
181/181 [==============================] - 36s 197ms/step - loss: 2.5084
В началото на сли запканската и псоделно чел дедпухдебата им Мектиченска „урекстврана са щъйзколе патън солянскава е доблвацие орищия стотрлия на лекотаратта а восъс зазегре воруи епрмуржажани стерцицивасте Осто н
Epoch 3/50
181/181 [==============================] - 36s 196ms/step - loss: 2.2684
В началото назайние на фи найфета продонява“ недо хогостия на серка пе-фюдиня на саматейлинят имгерствито що мугита) и презтоденатнета привиринестия угайския на репоратова мади „Флестандиа усу, т ускът Улушън 1000
Epoch 4/50
181/181 [==============================] - 36s 197ms/step - loss: 2.0585
В 

В началото на заради надружение.
Победута на бразилските сражения, 24 – С Кристрианския хидмерг навлизат на болестие, че същият значимо на следващия първия ѝ жента
21 март Лоскова́чева

Фолдовацьой срещу Нация с М
Epoch 14/50
181/181 [==============================] - 36s 197ms/step - loss: 1.3346
В началото на Либия води колониара.

Очестник не и&#кзарнограер участване на Хисан от независимост, във фланглед настолките, който обаче розниците му корпур и помстановистната ктавищ се върват със свадата на Луи и
Epoch 15/50
181/181 [==============================] - 36s 197ms/step - loss: 1.3121
В началото на австро-Унгарската армия той в двете страни в Дания са скличителни джецихи. Разгром границите на Рим просига списък и почти мирното големи военни колебия в Средорения с подправката и плащата на Стотен
Epoch 16/50
181/181 [==============================] - 36s 199ms/step - loss: 1.2916
В началото на 1578 г. според 6ктуст, Хавроайч, Войвопола от север маневрен кой като приема британска во

181/181 [==============================] - 36s 199ms/step - loss: 1.2731
В началото на материанците, 4,500 завладени се от България. Зелите на Кавкузия, 10-те неудължава е съмбата бижа да особено корабите заласти и едина. Скоро британски пътчи нанистите заявяват в Останската империя. Ф
Epoch 18/50
181/181 [==============================] - 36s 201ms/step - loss: 1.2551
В началото на 22 август 2097 г., който връщайки специалистките основате – Робуктор.

Първи засемващи групи за създаването на Лопес, в Бевруари Банафа.

След побеждувалото извън зима шведите ще въоръжени сили има т
Epoch 19/50
181/181 [==============================] - 36s 197ms/step - loss: 1.2392
В началото на 20 август в резултат на история, жената не Бардж Лукулфю извъзнали само издигането на кипасконтски коредски сили с унизинаити и корабите се джени разгромени, евреи и България на Балканите. Верналите 
Epoch 20/50
181/181 [==============================] - 36s 198ms/step - loss: 1.2235
В началото на Бащалис. В същот

В началото на Сърбия. След това, кака те смуски ожесточени боеве също така жертви (кото разстройлни цивили до 1980 тона етър основна шиша)
Борбата полковник Иран и Карсилората на Полша в страната е успешена Чаудос
Epoch 27/50
181/181 [==============================] - 36s 198ms/step - loss: 1.1355
В началото на 1941 година двете страни по правоприетическите сили като значителни войски в протестите. Бюканън приъзхвойки, статут.начинието на Константинопка на 29 март, да като сръбските сили Никола, Полша и Пак
Epoch 28/50
181/181 [==============================] - 36s 198ms/step - loss: 1.1243
В началото на 18 години е арентурни операции, населени организации, Олацо, Испания, Сквадните сили. Британците обучават померение е разрушено слаба официално в умедилионие да започне голяма част от пленени, което 
Epoch 29/50
181/181 [==============================] - 36s 198ms/step - loss: 1.1141
В началото на север на италия са прекъснено от клаузите на Португалската народна Панация и основата на 

В началото на таса две събития (1552) и в отговор на Румъния. По този победа за афсародическа операция с ръководено между свети да нанасят тежките единици по ниско положение в Македония. Москва и москови децасонск
Epoch 36/50
181/181 [==============================] - 36s 198ms/step - loss: 1.0461
В началото на войната (в ратова Кримска кондрика) са залежени, които са разположени на юг от страна на съставена в течение на съветските ловци на Гърция. Контрол на тази деца са русности. Като коментър Хайман Хоур
Epoch 37/50
181/181 [==============================] - 36s 198ms/step - loss: 1.0366
В началото на военните съюзници в демарсаците превземат столицата Каланя, образование пемотно обучентелни щати, за да плащат своите области от НАТО обстоятелства „война“, разовача „Опысва, значително предупреждава
Epoch 38/50
181/181 [==============================] - 36s 197ms/step - loss: 1.0274
В началото на фронта, назначава и некавишните оръжия, и по-скоро от напалението на Витхетри, в които Въ

В началото на Втората световна война е това взривяване, но членове на Втори корпус отново се състоящо с договора от Българското кралско съм град срещу Великобритания. Израелските разтова на сталинското правителств
Epoch 44/50
181/181 [==============================] - 36s 198ms/step - loss: 0.9761
В началото на при пътя на ценния конфликт в Древната реверени. Баки новия район от най-големите градове южно отдавна участие на Косово Вален и Кантакузин да така изненадано от населението е превзета на 11 декември
Epoch 45/50
181/181 [==============================] - 36s 201ms/step - loss: 0.9678
В началото на 1988 г., крал Ферис, който заедно може да присъединява кървавата къщата Правникска пехота за преговори, a 269 476 войника и 25 оръдия. Шестоточен сериал (Marein Latze), продължила към въстанието Севе
Epoch 46/50
181/181 [==============================] - 35s 196ms/step - loss: 0.9593
В началото на 55-те точка на хвойната е про-лата печели стара между двете войски води до съмнение позна

181/181 [==============================] - 36s 200ms/step - loss: 0.9517
В началото на опуактическо от „Лендина“ (Гигрев“. Ъгетите начело с Британската империя, Будая има множество стратегии в района на Фридрих II фон Филип прокомунистически настроенирански тайни - Си-ракстви „Войника 
Epoch 48/50
181/181 [==============================] - 36s 199ms/step - loss: 0.9435
В началото на вестник – Франсфахинето на Люнф решителна последна партизанска сила. Воденият кораб, налагането на афера с аграрната кампания влиза в развит, около 10 800 срмвите в Хузунон при включването с мирен до
Epoch 49/50
181/181 [==============================] - 36s 199ms/step - loss: 0.9358
В началото на германски президент до Лавказкия корпус в Европа на Оте предложение за безуспешно.

Естните участват в плодовете му. Триподъл капитал в Сръбско-българската война и Западна Българи от Балканския прото
Epoch 50/50
181/181 [==============================] - 36s 200ms/step - loss: 0.92872s - l


In [19]:
curr_model = build_model(1)
curr_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
curr_model.build(tf.TensorShape([1, None]))
print(generate_text(curr_model, "В началото на", 200))

В началото на България над Пирот; са разглебена и правителството като една от най-големите рибертори са пленени и едва през април 1921 г., излиза в крайна сметка натз. Деняма секреите действат на по-рда върху гърц
